In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.models import load_model

from src.nn_settings.gru_tflite_config import Config
from src.tflite_rnn import gru_functional, lstm_functional, tflite_lstm

/Users/i862304/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
keras_file = "../data/keras_rnn_1.h5"

config = Config()
data = config.data
num_input_characters = config.num_input_characters
num_output_characters = config.num_output_characters
input_encoding = config.input_encoding
output_encoding = config.output_encoding
enc = config.encoder
max_input_length = enc.max_input_length
max_output_length = enc.max_output_length

print("data_size", config.data_config.data_size)
print("char_string", config.data_config.char_string)

batch_size = config.batch_size
epochs = config.epochs
latent_dim = config.latent_dim

print("batch_size", batch_size)
print("epochs", epochs)
print("latent_dim", latent_dim)

print("\ninput chars", num_input_characters)
print("data size", len(data))
print(data[:5])



Creating Data set


/Users/i862304/workspace/NeuralNetworkPlayground/notebooks/src/dataset_builders/text_data.py:59: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  sample_data = np.vstack((get_one_sample() for _ in range(data_size)))


Building Vocabulary
Building Encoding
data_size 30000
char_string abcdeXY
batch_size 32
epochs 4
latent_dim 8

input chars 9
data size 30000
[['ecXdaYce' 'da']
 ['ecXbaYde' 'ba']
 ['dbXbcYaa' 'bc']
 ['deXeeYcd' 'ee']
 ['deXecYcd' 'ec']]


In [3]:
model = gru_functional(
           num_input_characters,
           num_output_characters,
           latent_dim,
           max_input_length,
           max_output_length,
          )

# model = lstm_functional(
#            num_input_characters,
#            num_output_characters,
#            latent_dim,
#            max_input_length,
#            max_output_length,
#           )

# model = tflite_lstm(
#            num_input_characters,
#            num_output_characters,
#            latent_dim,
#            max_input_length,
#            max_output_length,
#           )

print(model.summary())

model.fit(input_encoding, 
          output_encoding,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1)

model.save(keras_file)
# tf.keras.models.save_model(model, keras_file)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8, 9)              0         
_________________________________________________________________
gru (GRU)                    (None, 8)                 432       
_________________________________________________________________
repeat_vector (RepeatVector) (None, 2, 8)              0         
_________________________________________________________________
time_distributed (TimeDistri (None, 2, 9)              81        
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________
None
Train on 27000 samples, validate on 3000 samples
Epoch 1/4
27000/27000 [==============================] - 10s 355us/step - loss: 1.5899 - val_loss: 1.2772
Epoch 2/4
27000/27000 [==============================] - 8s 293us/step - loss: 1.0410 - val_loss: 0.7966
Epoch 3/4
2

In [4]:
## Demonstrate that the model does something
test_sample = "acXbdYea"
test_enc = enc.encode([test_sample], encode_as="input")
pred = model.predict(test_enc)
enc.decode(pred, decode_as="output")

array([b'bb'], dtype='|S2')

In [5]:
## Convert to TensorFlow Lite model.
## These docs describe the converter in the TensorFlow nightly release, installed using pip install tf-nightly
# converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_file)

## For tensorflow 1.12 it should be 
converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_file)


tflite_model = converter.convert()
open(tflite_file, "wb").write(tflite_model)

INFO:tensorflow:Froze 5 variables.
INFO:tensorflow:Converted 5 variables to const ops.


RuntimeError: TOCO failed see console for info.
b'2019-02-13 14:58:54.702642: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayV3\n2019-02-13 14:58:54.702712: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/TensorArray\n2019-02-13 14:58:54.702733: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayV3\n2019-02-13 14:58:54.702751: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/TensorArray_1\n2019-02-13 14:58:54.702847: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayScatterV3\n2019-02-13 14:58:54.702866: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3\n2019-02-13 14:58:54.702897: I tensorflow/contrib/lite/toco/impo

In [ ]:
ss = "RuntimeError: TOCO failed see console for info.\nb'2019-02-13 14:45:21.834947: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayV3\n2019-02-13 14:45:21.835013: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/TensorArray\n2019-02-13 14:45:21.835033: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayV3\n2019-02-13 14:45:21.835050: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/TensorArray_1\n2019-02-13 14:45:21.835121: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayScatterV3\n2019-02-13 14:45:21.835139: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3\n2019-02-13 14:45:21.835168: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835194: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835213: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835231: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835274: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835297: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835320: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: LoopCond\n2019-02-13 14:45:21.835332: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/while/LoopCond\n2019-02-13 14:45:21.835389: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayReadV3\n2019-02-13 14:45:21.835409: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835424: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.835440: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835458: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835475: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835489: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.835548: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835602: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835657: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835674: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835688: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.835738: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835790: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835842: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835859: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835872: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.835921: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.836051: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.836130: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayWriteV3\n2019-02-13 14:45:21.836145: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/while/TensorArrayWrite/TensorArrayWriteV3\n2019-02-13 14:45:21.836162: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.836176: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.836211: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Exit\n2019-02-13 14:45:21.836226: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Exit\n2019-02-13 14:45:21.836248: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayReadV3\n2019-02-13 14:45:21.839639: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before Removing unused ops: 102 operators, 178 arrays (0 quantized)\n2019-02-13 14:45:21.841691: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before general graph transformations: 102 operators, 178 arrays (0 quantized)\n2019-02-13 14:45:21.843801: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] After general graph transformations pass 1: 93 operators, 165 arrays (0 quantized)\n2019-02-13 14:45:21.971145: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] After general graph transformations pass 2: 92 operators, 164 arrays (0 quantized)\n2019-02-13 14:45:21.973872: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] After general graph transformations pass 3: 91 operators, 162 arrays (0 quantized)\n2019-02-13 14:45:21.975570: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before dequantization graph transformations: 91 operators, 162 arrays (0 quantized)\n2019-02-13 14:45:21.976779: F tensorflow/contrib/lite/toco/tooling_util.cc:618] Check failed: dim >= 1 (0 vs. 1)\n'"
print(ss)

# Check performance of tflite model

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.contrib.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
# change the following line to feed into your own data.
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)